# ModelSelection - Experiment
Notebook to experiment different model configurations and store results.

In [1]:
import os
from working_dir import set_wd
set_wd()
os.getcwd()

'/home/tales/ds/kaggle/football-match-prediction'

In [2]:
from pyspark.sql import SparkSession

spark = SparkSession.builder.config("spark.sql.debug.maxToStringFields", 500) \
                            .config("spark.driver.memory", "14g") \
                            .appName("ModelSelection-Experiment").getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

22/05/09 11:31:13 WARN Utils: Your hostname, tales-samsung resolves to a loopback address: 127.0.1.1; using 192.168.0.104 instead (on interface wlxd03745e80dbf)
22/05/09 11:31:13 WARN Utils: Set SPARK_LOCAL_IP if you need to bind to another address
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
22/05/09 11:31:14 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable


In [3]:
from datetime import datetime
import uuid
import pyspark.sql.functions as f
from pyspark.sql.functions import when
from src.dao import dao, dao_processed, dao_ml
from src.utils import dflib, stats, pretties, plot, plot_domain, palette

In [4]:
pretties.max_data_frame_columns()

# Loading Data

In [5]:
df_id = "8cf83372-54d4-4bf2-a1ee-5036ab694faa"

In [6]:
df_ttrain = dao_processed.load_processed_data(which_dataset="train_train", id=df_id, spark=spark)
df_tvalid = dao_processed.load_processed_data(which_dataset="train_valid", id=df_id, spark=spark)

metadata_json = dao_processed.load_processed_metadata(id=df_id)
print(f"df_ttrain shape: {dflib.shape(df_ttrain)}")
print(f"df_tvalid shape: {dflib.shape(df_tvalid)}")

df_ttrain shape: (87470, 14)
df_tvalid shape: (23468, 14)


# Fit Data Pipeline

In [7]:
from pyspark.ml import Pipeline, PipelineModel
from pyspark.ml.feature import VectorIndexer #Class for indexing categorical feature columns in a dataset of Vector.
from pyspark.ml.feature import VectorAssembler #A feature transformer that merges multiple columns into a vector column.
from pyspark.ml.feature import StringIndexer #A label indexer that maps a string column of labels to an ML column of label indices.
from pyspark.ml.feature import IndexToString
from src.ml.transformers import DropNaTransformer, UndersamplingTransformer

from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from src.ml import metrics

In [8]:
use_features = metadata_json["use_features"]

feature_assembler_transformer = VectorAssembler(inputCols=use_features, 
                                                outputCol="features")

target_indexer_transformer = StringIndexer(inputCol="target", 
                                           outputCol="target_indexed", 
                                           stringOrderType="alphabetDesc")
target_indexer_transformer = target_indexer_transformer.fit(df_ttrain)

inverter_transformer = IndexToString(inputCol="target_indexed", 
                                     outputCol="indexed_to_target", 
                                     labels=target_indexer_transformer.labels)

In [9]:
pipeline_stages=[DropNaTransformer(),
                 feature_assembler_transformer, 
                 target_indexer_transformer]

pipeline_model = PipelineModel(stages=pipeline_stages)

In [10]:
df_ttrain = pipeline_model.transform(df_ttrain.select(["target"] + use_features))
df_ttrain_undersampling = UndersamplingTransformer(target_colname="target").transform(df_ttrain)

df_tvalid = pipeline_model.transform(df_tvalid.select(["target"] + use_features))
df_tvalid_undersampling = UndersamplingTransformer(target_colname="target").transform(df_tvalid)

DropNaTransformer
UndersamplingTransformer


DropNaTransformer
UndersamplingTransformer


In [11]:
experiment_params = []

for missing_values_strategy in ["dropna"]:
    for undersampling in (True, False):
        for num_trees in (60, 25, 10):
            for max_depth in (15, 13, 10, 8, 5, 1, 0):
                for subsampling_rate in (0.8, 0.65, 0.5, 0.3):

                    params = {'missing_values_strategy': missing_values_strategy,
                              'undersampling': undersampling, 
                              'num_trees': num_trees, 
                              'max_depth': max_depth, 
                              'subsampling_rate': subsampling_rate}
                    
                    experiment_params.append(params)
                
print(f'n params: {len(experiment_params)}')

n params: 168


In [ ]:
counter = 0

for params in experiment_params:
    counter += 1
    if counter <= 84:
        continue
    print(f"n: {counter}")
    display(params)
    
    missing_values_strategy = params["missing_values_strategy"]
    undersampling = params["undersampling"]
    num_trees = params["num_trees"]
    max_depth = params["max_depth"]
    subsampling_rate = params["subsampling_rate"]
    
    if undersampling:
        use_df_train = df_ttrain_undersampling
        use_df_valid = df_tvalid_undersampling
    else:
        use_df_train = df_ttrain
        use_df_valid = df_tvalid
    
    ### TRAINING
    rf = RandomForestClassifier(labelCol="target_indexed", 
                                probabilityCol='proba', 
                                featuresCol="features", 
                                numTrees=num_trees,
                                maxDepth=max_depth,
                                subsamplingRate=subsampling_rate)
    
    model = rf.fit(use_df_train)
    
    
    ### TRAIN METRICS
    train_preds = model.transform(use_df_train)
    metrics_train_train = metrics.get_metrics(train_preds, 
                                                    labelCol="target_indexed", 
                                                    predictionCol="prediction", 
                                                    probabilityCol="proba")
    metrics_train_train["which_dataset"] = "train_train"
    
    ### VALIDATION METRICS
    valid_preds = model.transform(use_df_valid)
    metrics_train_valid = metrics.get_metrics(valid_preds, 
                                                    labelCol="target_indexed", 
                                                    predictionCol="prediction", 
                                                    probabilityCol="proba")
    metrics_train_valid["which_dataset"] = "train_valid"
    
    ### BULDING RESULTS
    id_model = str(uuid.uuid4())
#     result = {**params, **metric_values_train_train, **metric_values_train_valid}
    result = params
    result["metrics_train_train"] = metrics_train_train
    result["metrics_train_valid"] = metrics_train_valid
    result["id_model"] = id_model
    result["datetime"] = datetime.now().strftime("%Y-%m-%d %H:%M:%S")
    
    result = {**result, **params}
    
    print("accuracy train:", metrics_train_train["accuracy"])
    print("accuracy valid:", metrics_train_valid["accuracy"])
    print("log loss train:", metrics_train_train["log_loss"])
    print("log loss valid:", metrics_train_valid["log_loss"])

    dao_ml.save_result(result=result, id_data_build=df_id, model=model, id_model=id_model)
    
    pretties.hr()

n: 85


{'missing_values_strategy': 'dropna',
 'undersampling': False,
 'num_trees': 60,
 'max_depth': 15,
 'subsampling_rate': 0.8}

accuracy train: 0.7115096460011238
accuracy valid: 0.492748588797221
log loss train: 0.7743561909621416
log loss valid: 1.0129436194326276


n: 86


{'missing_values_strategy': 'dropna',
 'undersampling': False,
 'num_trees': 60,
 'max_depth': 15,
 'subsampling_rate': 0.65}

...

n: 132


{'missing_values_strategy': 'dropna',
 'undersampling': False,
 'num_trees': 25,
 'max_depth': 5,
 'subsampling_rate': 0.3}

accuracy train: 0.4943809702191422
accuracy valid: 0.494398610508033
log loss train: 1.019498042802492
log loss valid: 1.015462984388942


n: 133


{'missing_values_strategy': 'dropna',
 'undersampling': False,
 'num_trees': 25,
 'max_depth': 1,
 'subsampling_rate': 0.8}